# Clustering

## manual implementation of wssse method (not working, cannot extract distance from dense assembled vector)

In [ ]:
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt
from pyspark.sql.types import DoubleType, ArrayType
import pyspark.sql.functions as F
from pyspark.ml.linalg import Vectors, DenseVector

def calculate_WSSSE(df, prediction_col="prediction", features_col="features"):
    def squared_distance(feature_vec, mean_vec):
        if isinstance(feature_vec, DenseVector) and isinstance(mean_vec, DenseVector):
            return Vectors.squared_distance(feature_vec, mean_vec)
        else:
            return 0.0  # Return 0 if not valid vectors 
        # return Vectors.squared_distance(feature_vec, mean_vec)
    squared_distance_udf = F.udf(squared_distance, DoubleType())

    def extract_values(feature_vec):
        if isinstance(feature_vec, DenseVector): 
            return feature_vec.toArray().tolist() # Extract values directly
        else:
            return Vectors.dense(feature_vec.toArray().tolist())  

    extract_values_udf = F.udf(extract_values, ArrayType(DoubleType())) 

    subquery = df.select(prediction_col, F.explode(extract_values_udf(F.col(features_col))).alias('feature_value')) \
                 .groupBy(prediction_col, 'feature_value') \
                 .agg(F.mean('feature_value').alias('mean_features'))

    wssse = subquery.groupBy(prediction_col) \
                    .agg(F.sum(squared_distance_udf(F.col('feature_value'), F.col('mean_features'))).alias('wssse')) \
                    .select(F.sum('wssse')).collect()[0][0]

    return wssse

wssse = []

for k in [7, 8]: #because the beer styles are 8 so based on this features no more clusters should be created 
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features").setPredictionCol("prediction")
    predictions = kmeans.fit(df).transform(df)
    print(k)

    wssse.append(calculate_WSSSE(predictions))  # WSSSE calculation

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot([7, 8], wssse, 'bo-')  # Adjust 'k' values here too
plt.xlabel('Number of Clusters (k)')
plt.ylabel('WSSSE')
plt.title('Elbow Method for Optimal k')
plt.show()